In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import (
    train_test_split,
    cross_val_score,
    GridSearchCV
)
from sklearn.pipeline import Pipeline
from sklearn.linear_model import ARDRegression
from sklearn.decomposition import PCA, NMF
#from umap import UMAP
from sklearn.feature_selection import (
    mutual_info_regression,
    f_regression
)
from sklearn.feature_selection import SelectKBest
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import r2_score
from sklearn.preprocessing import (
    StandardScaler,
    MinMaxScaler
)
from sklearn.compose import TransformedTargetRegressor
from sklearn.preprocessing import (
    QuantileTransformer,
    quantile_transform
)
import csv
from sklearn.linear_model import TheilSenRegressor

In [2]:
SEED = 2024
TEST_SIZE = 0.25

In [3]:
np.random.seed(SEED)

In [4]:
data = pd.read_csv('data/all.csv')

In [5]:
y = data.values[:, 1].astype(np.float16)
x = data.values[:, 2:].astype(np.float16)

In [6]:
from scipy.signal import savgol_filter
from sklearn.base import BaseEstimator, TransformerMixin

class SavgolFilterTransformer(BaseEstimator, TransformerMixin):

    def __init__(self, window_length=2, polyorder=1):
        super(SavgolFilterTransformer, self).__init__()
        self.window_length = window_length
        self.polyorder = polyorder

    def fit(self, X, y=None):
        return self

    def transform(self, X, y=None):
        # Perform arbitary transformation
        X = savgol_filter(X, 
                          window_length=self.window_length, 
                          polyorder=self.polyorder)
        return X

In [10]:
def create_pipeline(regressor: BaseEstimator, mode: str = "naive"):
    if mode == "naive":
        filter_window_length = [4, 12]
        filter_poly_order = [3]
    elif mode == "full":
        filter_window_length = [4, 12, 24]
        filter_poly_order = [3]

    pipelines = [
        (
            {
                #"selection__k": [x.shape[1]//2, 2 * x.shape[1]//3],
                "scaler": [StandardScaler(), MinMaxScaler(feature_range=(0, 1))],
                "filter__window_length": filter_window_length,
                "filter__polyorder": filter_poly_order,
                "selection__k": [64, 48],
                "selection__score_func": [mutual_info_regression, f_regression],
                #"extraction__n_components": [x.shape[1]//3, x.shape[1]//4],
                "extraction": [PCA()],#, NMF(max_iter=10000)],
                "extraction__n_components": [32, 24]
            },
            Pipeline([
                ("scaler", "passthrough"),
                ("filter", SavgolFilterTransformer()),
                ("selection", SelectKBest()),
                ("extraction", "passthrough"),
                ("polynomial", PolynomialFeatures(include_bias=False, degree=1)),
                ("regression", regressor)
            ])
        ),
        (
            {
                #"selection__k": [x.shape[1]//2, 2 * x.shape[1]//3],
                "scaler": [StandardScaler(), MinMaxScaler(feature_range=(0, 1))],
                "filter__window_length": filter_window_length,
                "filter__polyorder": filter_poly_order,
                "selection__k": [64, 48],
                "selection__score_func": [mutual_info_regression, f_regression],
                #"extraction__n_components": [x.shape[1]//3, x.shape[1]//4],
                "extraction": [PCA()],#, NMF(max_iter=10000)],
                "extraction__n_components": [32, 24]
            },
            Pipeline([
                ("scaler", "passthrough"),
                ("filter", SavgolFilterTransformer()),
                ("selection", SelectKBest()),
                ("extraction", "passthrough"),
                ("polynomial", PolynomialFeatures(include_bias=False, degree=2)),
                ("regression", regressor)
            ])
        ),
        (
            {
                #"selection__k": [x.shape[1]//2, 2 * x.shape[1]//3],
                "scaler": [StandardScaler(), MinMaxScaler(feature_range=(0, 1))],
                "filter__window_length": filter_window_length,
                "filter__polyorder": filter_poly_order,
                "selection__k": [64, 48],
                "selection__score_func": [mutual_info_regression, f_regression],
                #"extraction__n_components": [x.shape[1]//3, x.shape[1]//4],
                "extraction": [PCA()],#, NMF(max_iter=10000)],
                "extraction__n_components": [32, 24]
            },
            Pipeline([
                ("scaler", "passthrough"),
                ("filter", SavgolFilterTransformer()),
                ("selection", SelectKBest()),
                ("extraction", "passthrough"),
                ("polynomial", PolynomialFeatures(include_bias=False, degree=3)),
                ("regression", regressor)
            ])
        ), (

    {
                #"selection__k": [x.shape[1]//2, 2 * x.shape[1]//3],
                "scaler": [StandardScaler(), MinMaxScaler(feature_range=(0, 1))],
                "selection__k": [64, 48],
                "selection__score_func": [mutual_info_regression, f_regression],
                #"extraction__n_components": [x.shape[1]//3, x.shape[1]//4],
                "extraction": [PCA()],#, NMF(max_iter=10000)],
                "extraction__n_components": [32, 24]
            },
            Pipeline([
                ("scaler", "passthrough"),
                ("selection", SelectKBest()),
                ("extraction", "passthrough"),
                ("polynomial", PolynomialFeatures(include_bias=False, degree=1)),
                ("regression", regressor)
            ])
        ),
        (
            {
                #"selection__k": [x.shape[1]//2, 2 * x.shape[1]//3],
                "scaler": [StandardScaler(), MinMaxScaler(feature_range=(0, 1))],
                "selection__k": [64, 48],
                "selection__score_func": [mutual_info_regression, f_regression],
                #"extraction__n_components": [x.shape[1]//3, x.shape[1]//4],
                "extraction": [PCA()],#, NMF(max_iter=10000)],
                "extraction__n_components": [32, 24]
            },
            Pipeline([
                ("scaler", "passthrough"),
                ("selection", SelectKBest()),
                ("extraction", "passthrough"),
                ("polynomial", PolynomialFeatures(include_bias=False, degree=2)),
                ("regression", regressor)
            ])
        ),
        (
            {
                #"selection__k": [x.shape[1]//2, 2 * x.shape[1]//3],
                "scaler": [StandardScaler(), MinMaxScaler(feature_range=(0, 1))],
                "selection__k": [64, 48],
                "selection__score_func": [mutual_info_regression, f_regression],
                #"extraction__n_components": [x.shape[1]//3, x.shape[1]//4],
                "extraction": [PCA()],#, NMF(max_iter=10000)],
                "extraction__n_components": [32, 24],
            },
            Pipeline([
                ("scaler", "passthrough"),
                ("selection", SelectKBest()),
                ("extraction", "passthrough"),
                ("polynomial", PolynomialFeatures(include_bias=False, degree=3)),
                ("regression", regressor)
            ])
        ),
    ]

    return pipelines

In [11]:
CVS = [2, 3]

In [12]:
def optimize_pipeline(cv, pipeline, x_train, x_test, y_train, y_test):
        param_grid = pipeline[0]
        pipe = pipeline[1]
        print(param_grid)
        print(pipe)
        grid_search = GridSearchCV(
            pipe,
            param_grid=param_grid,
            scoring="r2",
            cv=cv,
            refit=True,
            n_jobs=-1,
            verbose=1
        )
        grid_search.fit(x_train, y_train)
        y_pred = grid_search.best_estimator_.predict(x_test)
        train_score = r2_score(y_train, grid_search.best_estimator_.predict(x_train))
        test_score = r2_score(y_test, y_pred)
        scores = cross_val_score(
            grid_search.best_estimator_, x_test, y_test, scoring="r2", cv=3
        )
        print(f"Best Score: {grid_search.best_score_:.3f}; R2 Score: Train = {train_score:.3f}; Test = {test_score:.3f}")
        print(grid_search.best_params_)

        print("")
        #print(grid_search.cv_results_)
        pipeline_with_trans_target = TransformedTargetRegressor(
            regressor=grid_search.best_estimator_,
            transformer=QuantileTransformer(n_quantiles=900, output_distribution="normal")
        )
        pipeline_with_trans_target.fit(x_train, y_train)
        y_pred = pipeline_with_trans_target.predict(x_test)
        train_score_trans = r2_score(y_train, pipeline_with_trans_target.predict(x_train))
        test_score_trans = r2_score(y_test, y_pred)
        trans_scores = cross_val_score(
            pipeline_with_trans_target, x_test, y_test, scoring="r2", cv=3
        )
        print(f"(Transformed Target) R2 Score: Train = {train_score_trans:.3f}; Test = {test_score_trans:.3f}")

        return [round(grid_search.best_score_, 3), 
                round(train_score, 3),
                round(test_score, 3),
                round(scores.mean(), 3),
                round(scores.std(), 3),
                round(train_score_trans, 3),
                round(test_score_trans, 3),
                round(trans_scores.mean(), 3),
                round(trans_scores.std(), 3)
                ], grid_search.best_params_

In [ ]:
with open("results.csv", 'w') as csvfile:
    csvwriter = csv.writer(csvfile, quoting=csv.QUOTE_ALL)
    for num_experiment in range(1, 5):
        x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=TEST_SIZE,
                                                            shuffle=True,
                                                            random_state=SEED//num_experiment)
        y_train_trans = quantile_transform(
            y_train.reshape(-1, 1), n_quantiles=900, output_distribution="normal", copy=True
        ).squeeze()

        y_test_trans = quantile_transform(
            y_test.reshape(-1, 1), n_quantiles=900, output_distribution="normal", copy=True
        ).squeeze()

        for cv in CVS:
            for ix_regressor, regressor in enumerate([ARDRegression(), TheilSenRegressor()]):
                pipelines = create_pipeline(regressor, mode="full")
                for ix_pipeline, pipeline in enumerate(pipelines):
                    try:
                        print(f"CV: {cv}")

                        results, best_params = (
                            optimize_pipeline(cv, pipeline, x_train, x_test, y_train, y_test)
                        )
                        if ix_pipeline in [0, 1, 2]:
                            pipeline_polyorder = best_params["filter__polyorder"]
                            pipeline_window_length = best_params["filter__window_length"]
                        else:
                            pipeline_polyorder = 0
                            pipeline_window_length = 0

                        hyperparams = [pipeline_polyorder,
                                    pipeline_window_length, 
                                    best_params["scaler"], 
                                    best_params["selection__k"],
                                    best_params["selection__score_func"],
                                    best_params["extraction"],
                                    best_params["extraction__n_components"]]

                        trans_results, trans_best_params = (
                            optimize_pipeline(cv, pipeline, x_train, x_test, y_train_trans, y_test_trans)
                        )

                        if ix_pipeline in [0, 1, 2]:
                            trans_pipeline_polyorder = trans_best_params["filter__polyorder"]
                            trans_pipeline_window_length = trans_best_params["filter__window_length"]
                        else:
                            trans_pipeline_polyorder = 0
                            trans_pipeline_window_length = 0

                        trans_hyperparams = [trans_pipeline_polyorder,
                                            trans_pipeline_window_length, 
                                            trans_best_params["scaler"], 
                                            trans_best_params["selection__k"],
                                            trans_best_params["selection__score_func"],
                                            trans_best_params["extraction"],
                                            trans_best_params["extraction__n_components"],]

                        csvwriter.writerow([
                            num_experiment,
                            cv,
                            ix_regressor,
                            ix_pipeline,
                            *hyperparams,
                            *results,
                            *trans_hyperparams,
                            *trans_results
                        ])
                        csvfile.flush()
                        print("="*50)
                    except Exception as e:
                        print(e)

CV: 2
{'scaler': [StandardScaler(), MinMaxScaler()], 'filter__window_length': [4, 12, 24], 'filter__polyorder': [3], 'selection__k': [64, 48], 'selection__score_func': [<function mutual_info_regression at 0x7f139b130c10>, <function f_regression at 0x7f139b132830>], 'extraction': [PCA()], 'extraction__n_components': [32, 24]}
Pipeline(steps=[('scaler', 'passthrough'),
                ('filter', SavgolFilterTransformer()),
                ('selection', SelectKBest()), ('extraction', 'passthrough'),
                ('polynomial',
                 PolynomialFeatures(degree=1, include_bias=False)),
                ('regression', ARDRegression())])
Fitting 2 folds for each of 48 candidates, totalling 96 fits


/home/borjaeg/anaconda3/envs/multispectral/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:2663: UserWarning: n_quantiles (900) is greater than the total number of samples (771). n_quantiles is set to n_samples.
  warnings.warn(
/home/borjaeg/anaconda3/envs/multispectral/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:2663: UserWarning: n_quantiles (900) is greater than the total number of samples (257). n_quantiles is set to n_samples.
  warnings.warn(


Best Score: 0.458; R2 Score: Train = 0.514; Test = 0.456
{'extraction': PCA(), 'extraction__n_components': 32, 'filter__polyorder': 3, 'filter__window_length': 4, 'scaler': MinMaxScaler(), 'selection__k': 48, 'selection__score_func': <function mutual_info_regression at 0x7f139b130c10>}



/home/borjaeg/anaconda3/envs/multispectral/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:2663: UserWarning: n_quantiles (900) is greater than the total number of samples (771). n_quantiles is set to n_samples.
  warnings.warn(
/home/borjaeg/anaconda3/envs/multispectral/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:2663: UserWarning: n_quantiles (900) is greater than the total number of samples (171). n_quantiles is set to n_samples.
  warnings.warn(
/home/borjaeg/anaconda3/envs/multispectral/lib/python3.10/site-packages/sklearn/compose/_target.py:190: UserWarning: The provided functions or transformer are not strictly inverse of each other. If you are sure you want to proceed regardless, set 'check_inverse=False'
  warnings.warn(
/home/borjaeg/anaconda3/envs/multispectral/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:2663: UserWarning: n_quantiles (900) is greater than the total number of samples (171). n_quantiles is set to n_samples.
  warnin

(Transformed Target) R2 Score: Train = 0.241; Test = 0.128
{'scaler': [StandardScaler(), MinMaxScaler()], 'filter__window_length': [4, 12, 24], 'filter__polyorder': [3], 'selection__k': [64, 48], 'selection__score_func': [<function mutual_info_regression at 0x7f139b130c10>, <function f_regression at 0x7f139b132830>], 'extraction': [PCA()], 'extraction__n_components': [32, 24]}
Pipeline(steps=[('scaler', 'passthrough'),
                ('filter', SavgolFilterTransformer()),
                ('selection', SelectKBest()), ('extraction', 'passthrough'),
                ('polynomial',
                 PolynomialFeatures(degree=1, include_bias=False)),
                ('regression', ARDRegression())])
Fitting 2 folds for each of 48 candidates, totalling 96 fits
Best Score: 0.534; R2 Score: Train = 0.589; Test = 0.556
{'extraction': PCA(), 'extraction__n_components': 32, 'filter__polyorder': 3, 'filter__window_length': 12, 'scaler': StandardScaler(), 'selection__k': 64, 'selection__score_func'

...

(Transformed Target) R2 Score: Train = 0.594; Test = 0.572
CV: 3
{'scaler': [StandardScaler(), MinMaxScaler()], 'filter__window_length': [4, 12, 24], 'filter__polyorder': [3], 'selection__k': [64, 48], 'selection__score_func': [<function mutual_info_regression at 0x7f139b130c10>, <function f_regression at 0x7f139b132830>], 'extraction': [PCA()], 'extraction__n_components': [32, 24]}
Pipeline(steps=[('scaler', 'passthrough'),
                ('filter', SavgolFilterTransformer()),
                ('selection', SelectKBest()), ('extraction', 'passthrough'),
                ('polynomial', PolynomialFeatures(include_bias=False)),
                ('regression', ARDRegression())])
Fitting 3 folds for each of 48 candidates, totalling 144 fits
Best Score: 0.712; R2 Score: Train = 0.869; Test = 0.766
{'extraction': PCA(), 'extraction__n_components': 24, 'filter__polyorder': 3, 'filter__window_length': 4, 'scaler': MinMaxScaler(), 'selection__k': 48, 'selection__score_func': <function mutual_info

/home/borjaeg/anaconda3/envs/multispectral/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:2663: UserWarning: n_quantiles (900) is greater than the total number of samples (771). n_quantiles is set to n_samples.
  warnings.warn(
/home/borjaeg/anaconda3/envs/multispectral/lib/python3.10/site-packages/sklearn/compose/_target.py:190: UserWarning: The provided functions or transformer are not strictly inverse of each other. If you are sure you want to proceed regardless, set 'check_inverse=False'
  warnings.warn(
/home/borjaeg/anaconda3/envs/multispectral/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:2663: UserWarning: n_quantiles (900) is greater than the total number of samples (171). n_quantiles is set to n_samples.
  warnings.warn(
/home/borjaeg/anaconda3/envs/multispectral/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:2663: UserWarning: n_quantiles (900) is greater than the total number of samples (171). n_quantiles is set to n_samples.
  warnin

(Transformed Target) R2 Score: Train = 0.790; Test = 0.447
{'scaler': [StandardScaler(), MinMaxScaler()], 'filter__window_length': [4, 12, 24], 'filter__polyorder': [3], 'selection__k': [64, 48], 'selection__score_func': [<function mutual_info_regression at 0x7f139b130c10>, <function f_regression at 0x7f139b132830>], 'extraction': [PCA()], 'extraction__n_components': [32, 24]}
Pipeline(steps=[('scaler', 'passthrough'),
                ('filter', SavgolFilterTransformer()),
                ('selection', SelectKBest()), ('extraction', 'passthrough'),
                ('polynomial', PolynomialFeatures(include_bias=False)),
                ('regression', ARDRegression())])
Fitting 3 folds for each of 48 candidates, totalling 144 fits
Best Score: 0.643; R2 Score: Train = 0.767; Test = 0.671
{'extraction': PCA(), 'extraction__n_components': 24, 'filter__polyorder': 3, 'filter__window_length': 12, 'scaler': MinMaxScaler(), 'selection__k': 64, 'selection__score_func': <function mutual_info_regr

/home/borjaeg/anaconda3/envs/multispectral/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:2663: UserWarning: n_quantiles (900) is greater than the total number of samples (771). n_quantiles is set to n_samples.
  warnings.warn(
/home/borjaeg/anaconda3/envs/multispectral/lib/python3.10/site-packages/sklearn/compose/_target.py:190: UserWarning: The provided functions or transformer are not strictly inverse of each other. If you are sure you want to proceed regardless, set 'check_inverse=False'
  warnings.warn(
/home/borjaeg/anaconda3/envs/multispectral/lib/python3.10/site-packages/sklearn/preprocessing/_data.py:2663: UserWarning: n_quantiles (900) is greater than the total number of samples (171). n_quantiles is set to n_samples.
  warnings.warn(
/home/borjaeg/anaconda3/envs/multispectral/lib/python3.10/site-packages/sklearn/compose/_target.py:190: UserWarning: The provided functions or transformer are not strictly inverse of each other. If you are sure you want to proceed r

(Transformed Target) R2 Score: Train = 0.767; Test = 0.671
CV: 3
{'scaler': [StandardScaler(), MinMaxScaler()], 'filter__window_length': [4, 12, 24], 'filter__polyorder': [3], 'selection__k': [64, 48], 'selection__score_func': [<function mutual_info_regression at 0x7f139b130c10>, <function f_regression at 0x7f139b132830>], 'extraction': [PCA()], 'extraction__n_components': [32, 24]}
Pipeline(steps=[('scaler', 'passthrough'),
                ('filter', SavgolFilterTransformer()),
                ('selection', SelectKBest()), ('extraction', 'passthrough'),
                ('polynomial',
                 PolynomialFeatures(degree=3, include_bias=False)),
                ('regression', ARDRegression())])
Fitting 3 folds for each of 48 candidates, totalling 144 fits
